In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/mmALL_073120_csv.csv')

In [3]:
df.head()

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...
2,201990003,Canada,20,1990,North America,1,3,1.0,7.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai..."
3,201990004,Canada,20,1990,North America,1,4,12.0,7.0,1990.0,...,NaN,accomodation,NaN,NaN,NaN,NaN,NaN,NaN,1. indians gather as siege intensifies; armed ...,canada s federal government has agreed to acqu...
4,201990005,Canada,20,1990,North America,1,5,14.0,8.0,1990.0,...,NaN,crowd dispersal,arrests,accomodation,NaN,NaN,NaN,NaN,1. dozens hurt in mohawk blockade protest the ...,protests were directed against the state due t...


In [4]:
# I will be removing the current columns for state response and replacing them with columns
# containing binary data for each type of response. First, I will create a new column which
# will contain whatever the final state response for each protest was, in case we want to
# use that for something or need more chronological information.

responses = df['stateresponse1'].value_counts().index.tolist()
responses

['ignore',
 'crowd dispersal',
 'arrests',
 'accomodation',
 'shootings',
 'beatings',
 'killings']

In [5]:
for i in range(7,0,-1):
    df['final'] = 0
    for ind in range(df.shape[0]):
        if df[f'stateresponse{i}'][ind] in responses:
            df['final'][ind] = df[f'stateresponse{i}'][ind]

<ipython-input-5-836a05d1a1e3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final'][ind] = df[f'stateresponse{i}'][ind]
/Users/ncantle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
# I've seen this warning before, but I don't really understand it and don't think it's
# much of a problem? Not totally sure.

for item in responses:
    df[f'{item}'] = 0
    for i in range(1,8):
        for ind in range(df.shape[0]):
            if df[f'stateresponse{i}'][ind] == f'{item}':
                df[f'{item}'][ind] = 1

<ipython-input-6-fb033b7d2ab3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{item}'][ind] = 1


In [7]:
df.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protest', 'protestnumber',
       'startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes', 'final', 'ignore', 'crowd dispersal', 'arrests',
       'accomodation', 'shootings', 'beatings', 'killings'],
      dtype='object')

In [8]:
# We will use the new binary columns instead of these original columns. This takes care of
# all the nulls we had and will let us actually use these as target variables. This way, 
# we can run some binary regression (i.e. "success"/"no success", "killings"/"no killings")
# and also multiclass regressions where we can include all of the government responses
# together as the target.

df.drop(columns = ['stateresponse1','stateresponse2','stateresponse3',
                  'stateresponse4','stateresponse5','stateresponse6',
                  'stateresponse7'], inplace = True)

In [9]:
# I'm going to drop the "ccode" column, as this is an arbitrary numberical tag applied to
# each country and doesn't actually offer any insight. Also dropping the 'id' column since
# this is essentially the same information that the date columns are providing.

df.drop(columns = ['ccode', 'id'], inplace = True)

In [10]:
df.isnull().sum()

country                      0
year                         0
region                       0
protest                      0
protestnumber                0
startday                  1906
startmonth                1906
startyear                 1906
endday                    1906
endmonth                  1906
endyear                   1906
protesterviolence         1387
location                  1927
participants_category     7258
participants              1399
protesteridentity         2461
protesterdemand1          1907
protesterdemand2         14168
protesterdemand3         16762
protesterdemand4         16314
sources                   1910
notes                     1952
final                        0
ignore                       0
crowd dispersal              0
arrests                      0
accomodation                 0
shootings                    0
beatings                     0
killings                     0
dtype: int64

In [11]:
# Creating the same for loop to create binary columns for the protestor demands instead
# of what we currently have. Using basically the same technique as before. These columns
# will be really interesting to compare to the state responses (for example, do protests
# over the removal of a politician result in killings more than a labor wage dispute?)

demands = df['protesterdemand1'].value_counts().index.tolist()

for item in demands:
    df[f'{item}'] = 0
    for i in range(1,5):
        for ind in range(df.shape[0]):
            if df[f'protesterdemand{i}'][ind] == f'{item}':
                df[f'{item}'][ind] = 1

<ipython-input-11-8e1ec7366244>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{item}'][ind] = 1


In [12]:
# Dropping the original columns which we no longer need.

df.drop(columns = ['protesterdemand1','protesterdemand2',
                  'protesterdemand3','protesterdemand4'], inplace = True)

In [13]:
df.isnull().sum()

country                           0
year                              0
region                            0
protest                           0
protestnumber                     0
startday                       1906
startmonth                     1906
startyear                      1906
endday                         1906
endmonth                       1906
endyear                        1906
protesterviolence              1387
location                       1927
participants_category          7258
participants                   1399
protesteridentity              2461
sources                        1910
notes                          1952
final                             0
ignore                            0
crowd dispersal                   0
arrests                           0
accomodation                      0
shootings                         0
beatings                          0
killings                          0
political behavior, process       0
labor wage dispute          

In [14]:
# I'm just going to remove nulls in the "notes" column and see if that takes care of most
# of the nulls in the dataframe. Just from looking at it, it looks as though the nulls
# are basically contained in the same rows.

df = df.dropna(axis=0, subset=['notes'])

In [15]:
df.isnull().sum()

country                           0
year                              0
region                            0
protest                           0
protestnumber                     0
startday                          0
startmonth                        0
startyear                         0
endday                            0
endmonth                          0
endyear                           0
protesterviolence                 0
location                         21
participants_category          5308
participants                     12
protesteridentity               554
sources                           4
notes                             0
final                             0
ignore                            0
crowd dispersal                   0
arrests                           0
accomodation                      0
shootings                         0
beatings                          0
killings                          0
political behavior, process       0
labor wage dispute          

In [16]:
# As expected, the vast majority of nulls came from the same place. Getting rid of rows
# that had nulls in the "notes" column took care of most of the nulls in the dataset.

In [17]:
# After looking at the manual, I'm thinking it will make sense to remove the 
# 'participants_category' column. This is basically a column that puts the "participants"
# into buckets of various sizes. We will likely be more interested in the actual number
# of participants as opposed to the range, and I am confused at the huge number of nulls
# in the data for this categorical variable they created. If we decide to analyze the size
# of a protest with buckets, we can easily make them ourselves without including 5000+
# nulls.

df.drop(columns = 'participants_category', inplace = True)

In [18]:
# I will leave the small number of nulls in for this column.

df['protesteridentity'].value_counts()

protesters                                 1541
students                                    642
workers                                     272
unspecified                                 224
farmers                                     219
                                           ... 
protesters against president kabila           1
farmers and rural workers                     1
force ouvriere union                          1
greenpeace and emma and sophie thompson       1
moldovans, opposition supporters              1
Name: protesteridentity, Length: 5768, dtype: int64

In [19]:
# This is a really weird column with some really important information. Either we will find
# a way to use this as categorical data or we will have to figure out a way to make this
# more numerical. I'm going to leave it for now.

df['participants'].value_counts()

100s                         1328
1000s                        1186
50+                          1138
50                            998
1000                          966
                             ... 
Huge protests                   1
28500                           1
300s                            1
> 70000                         1
More than 200 campaigners       1
Name: participants, Length: 781, dtype: int64

In [20]:
df['accomodation']

0        0
1        0
2        0
3        1
4        1
        ..
17136    0
17138    0
17139    1
17140    0
17141    0
Name: accomodation, Length: 15193, dtype: int64

At this point, I have created the necessary target variable columns, cleaned up the majority of the nulls, and have turned some categorical data into useful binary columns. I'm going to save this dataframe into the data folder, then I will save a dataframe for each region. This should be a good place to start.

In [24]:
df.to_csv('../data/global.csv', index = False)

In [25]:
regions = df['region'].value_counts().index.tolist()
regions

['Europe',
 'Africa',
 'Asia',
 'South America',
 'MENA',
 'North America',
 'Central America',
 'Oceania']

In [26]:
for item in regions:
    snake_item = item.replace(' ','_').lower()
    df[df['region'] == item].to_csv(f'../data/{snake_item}.csv', index = False)